### Option Strategy Evaluation (Module 1)
Calculate returns across all permutations for various option strategies using the SPY BigQuery Dataset. The results can then be compared using a Data Visualization Tool to determine the optimal place at which to position strike prices. Returns will be calculated for the following strategies.
- Buy Call
- Buy Put
- Buy Call Spread
- Buy Put Spread
- Write Call Spread
- Write Put Spread
- Write Iron Condor

The holding period for all calculations will be 1 week and the expiry date will be 1 week out as well.

The above option strategies fall into 1 of 3 categories; Bullish, Bearish and Neutral. Choosing a strategy therefore implies an expectation as to the future behavior of the underlying stock. In order to evaluate trades in the context of this expectation, we need to identify the direction in which the market moved for each week in the study. The market_direction field will be derived as follows using the At-The-Money Implied Volatility.
- Strong Up - up more than the average expected move (upper 25% of a Normal Distribution)
- Neutral Up - up less than the average expected move (between 0% and upper 25% of a Normal Distribution)
- Neutral Down - down less than the average expected move (between 0% and lower 25% of a Normal Distribution)
- Strong Down - down more than the average expected move (lower 25% of a Normal Distribution)

The BigQuery SPY table has a sampling_key field (random number between 0 and 1) associated with each quote record which will help us determine the various returns that can be expected based on the accuracy of a model. To illustrate, consider a binary model that predicts SPY moving either up or down with a 70% accuracy rate. We could simulate this model by having it make correct predictions 70% of the time. In other words, we would have our simulated model predict that SPY will move up if it did in fact move up and the quote record had a sampling_key greater than .3. Likewise our model would predict that SPY will move down if it did in fact move down and the quote record had a sampling_key greater than .3 (note: 70% of random numbers betweeen 0 and 1 will have a value greater than .3). Using this approach we can simulate the expected returns for arbitrary models with various prediction acurracy rates.

### Output File Data Elements
- entry_date
- exit_date
- expiry_date
- strike_price
- entry_stock_price
- entry_atm_price
- entry_atm_iv
- entry_call_bid
- entry_call_ask
- entry_call_moneyness
- entry_put_bid
- entry_put_ask
- entry_put_moneyness
- exit_stock_price
- exit_call_price
- exit_put_price
- market_direction
- sampling_key

In [2]:
#assign dependencies and constants
import pandas as pd
import csv
import datetime
import math
from decimal import Decimal

### BigQuery SQL Statements

In [3]:
%%bigquery bq_spy_1wk
-- weekly option entry and exit quotes; 1 week holding period; 1 week to expiry
with trade_entry as
(
select quote_week as entry_week, quote_date as entry_date, expiry_date, days_to_expiry, strike_price, 
  call_bid as entry_call_bid, call_ask as entry_call_ask, call_moneyness as entry_call_moneyness, 
  put_bid as entry_put_bid, put_ask as entry_put_ask, put_moneyness as entry_put_moneyness, 
  underlying_price as entry_stock_price, atm_price as entry_atm_price, atm_iv as entry_atm_iv, sampling_key
from expiry-week.option_quotes.SPY, UNNEST(expiry_dates), UNNEST(strike_prices)
)
select t1.*, t2.quote_date as exit_date, t2.quote_week as exit_week, t2.underlying_price as exit_stock_price
from trade_entry t1
inner join expiry-week.option_quotes.SPY t2
  on t1.entry_week + 1 = t2.quote_week
    and t1.expiry_date = t2.quote_date
order by t1.entry_week


### Create Data File for calculating Strategy Returns

In [4]:
def derive_market_direction(entry_atm_price, entry_atm_iv, exit_stock_price, days_to_expiry):
    """
    get the direction in which the market moved
    """
    #calculate interquartile range price boundaries
    upper_boundary = entry_atm_price * math.exp(entry_atm_iv * .6745 * math.sqrt(days_to_expiry / 365))
    lower_boundary = entry_atm_price * math.exp(-entry_atm_iv * .6745 * math.sqrt(days_to_expiry / 365))

    if exit_stock_price > upper_boundary:
        return 'Strong Up'
    
    if exit_stock_price < lower_boundary:
        return 'Strong Down'
    
    if exit_stock_price >= entry_atm_price:
        return 'Neutral Up'  
   
    return 'Neutral Down'  
   

In [5]:
def spy_1wk_options():
    """
    generate evaluation file for options expirng in 1 week
    since these options will have expired we will use their intrinsic values in place of exit bid and ask prices
    """
    def call_intrinsic_value(strike_price, stock_price):
        if strike_price > stock_price:
            return 0
        else:
            return round(stock_price - strike_price, 2)
    
    def put_intrinsic_value(strike_price, stock_price):
        if strike_price < stock_price:
            return 0
        else:
            return round(strike_price - stock_price, 2)
    
    #make copy of bigquery generated dataframe and convert objects to proper data types
    #note that bigquer returns numeric and date data types as objects
    data_types = {'entry_date': 'datetime64', 'expiry_date': 'datetime64', 'strike_price': 'float64',
    'entry_call_bid': 'float64', 'entry_call_ask': 'float64', 'entry_call_moneyness': 'float64',
    'entry_put_bid': 'float64', 'entry_put_ask': 'float64', 'entry_put_moneyness': 'float64',
    'entry_stock_price': 'float64', 'entry_atm_price': 'float64', 'entry_atm_iv': 'float64',
    'sampling_key': 'float64', 'exit_date': 'datetime64', 'exit_stock_price': 'float64'}
     
    df = bq_spy_1wk.astype(data_types)
    
    #calculated fields
    df['call_intrinsic_value'] = df.apply(lambda x: call_intrinsic_value(x['strike_price'], x['exit_stock_price']), axis=1)
    df['put_intrinsic_value'] = df.apply(lambda x: put_intrinsic_value(x['strike_price'], x['exit_stock_price']), axis=1)
    df['market_direction'] = df.apply(lambda x: derive_market_direction(x['entry_atm_price'], x['entry_atm_iv'], x['exit_stock_price'], x['days_to_expiry']), axis=1)
    
    #create evaluation file
    with open('data/spy_1wk_options.csv', 'w', newline='') as f:
        
        out_csv = csv.writer(f, lineterminator='\n')
        
        columns = ['entry_date', 'exit_date', 'expiry_date','strike_price', 'entry_stock_price', 'entry_atm_price', 'entry_atm_iv', 
            'entry_call_bid', 'entry_call_ask', 'entry_call_moneyness', 'entry_put_bid', 'entry_put_ask', 'entry_put_moneyness',
            'exit_stock_price', 'exit_call_price', 'exit_put_price','market_direction', 'sampling_key']
        
        out_csv.writerow(columns)
       
        for row in df.itertuples():
            record = [
                datetime.datetime.date(row.entry_date),
                datetime.datetime.date(row.exit_date),
                datetime.datetime.date(row.expiry_date),
                round(row.strike_price, 2),           
                round(row.entry_stock_price, 2),
                round(row.entry_atm_price, 2),
                round(row.entry_atm_iv, 3),
                round(row.entry_call_bid, 2),
                round(row.entry_call_ask, 2),           
                round(row.entry_call_moneyness, 3),
                round(row.entry_put_bid, 2),
                round(row.entry_put_ask, 2),  
                round(row.entry_put_moneyness, 3),
                round(row.exit_stock_price, 2),
                round(row.call_intrinsic_value, 2),
                round(row.put_intrinsic_value, 2),
                row.market_direction,
                round(row.sampling_key, 3)]    
            out_csv.writerow(record)

#execute function
spy_1wk_options()                                                                                                  
     